# Adecuación de una BBDD con coordenadas para ejecutar geoqueries

#### Tomamos la BBDD del fichero JSON comprimido en la carpeta 'inputs' y la adecuamos para que contenga un fila por cada una de las oficinas que poseen las empresas del dataset, y además añadimos una columna con el formato adecuado para ejecutar geoqueries en MongoDB

Importamos librerías

In [1]:
from pymongo import MongoClient
import pandas as pd
from src.gameslib import (geocode,gMapsLink,toCartoPoint,toGeoJSON)
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re

Indicamos la base de datos de MongoDB (en carpeta /output, unzip y cargar en servidor local mongodb)

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

Tomamos una selección de campos para la nueva colección.

In [3]:

select = {"name":1,"category_code":1,"number_of_employees":1,"tag_list":1,"offices":1, "_id":0}

companies2 = list(db["companies"].find({},select))


In [4]:

df = pd.DataFrame(companies2)
df.head()

,name,category_code,number_of_employees,tag_list,offices
0,AdventNet,enterprise,600.0,,"[{'description': 'Headquarters', 'address1': '..."
1,Zoho,software,1600.0,"zoho, officesuite, spreadsheet, writer, projec...","[{'description': 'Headquarters', 'address1': '..."
2,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...","[{'description': '', 'address1': '710 - 2nd Av..."
3,Omnidrive,network_hosting,NaN,"storage, sharing, edit, online","[{'description': '', 'address1': 'Suite 200', ..."
4,Postini,web,NaN,,"[{'description': None, 'address1': '959 Skyway..."


Como vemos arriba, hay que *desmembrar* la columna de offices para sacar de la lista de diccionarios las coordenadas de las distintas ubicaciones

In [5]:
#explode and expand
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

Drop NaN in coords

In [6]:
cleanData.dropna(subset=['latitude','longitude'],inplace=True)
cleanData.reset_index(inplace=True,drop=True)
cleanData.drop(columns=["offices"],inplace=True)

cleanData.fillna(value={'number_of_employees':5.0},inplace=True)
cleanData.head()

,name,category_code,number_of_employees,tag_list,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,enterprise,600.0,,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Zoho,software,1600.0,"zoho, officesuite, spreadsheet, writer, projec...",Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
4,Postini,web,5.0,,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


Con la longitud y latitud limpias y sin nulos, creamos una columna tipo POINT para cartoframe y otra tipo geoJSON para incorporar a MongoDB

In [7]:
toCartoPoint(-0.116537,51.517742)

'POINT (-0.116537 51.517742)'

In [8]:
toGeoJSON(-0.116537,51.517742)

{'type': 'Point', 'coordinates': [-0.116537, 51.517742]}

In [9]:
cleanData["geometry"] = cleanData[["latitude","longitude"]].apply(lambda x:toCartoPoint(x.longitude,x.latitude), axis=1)
cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:toGeoJSON(x.longitude,x.latitude), axis=1)
cleanData[["latitude","longitude","geometry","location"]].head()

,latitude,longitude,geometry,location
0,37.692934,-121.904945,POINT (-121.904945 37.692934),"{'type': 'Point', 'coordinates': [-121.904945,..."
1,37.692934,-121.904945,POINT (-121.904945 37.692934),"{'type': 'Point', 'coordinates': [-121.904945,..."
2,47.603122,-122.333253,POINT (-122.333253 47.603122),"{'type': 'Point', 'coordinates': [-122.333253,..."
3,40.723731,-73.996431,POINT (-73.9964312 40.7237306),"{'type': 'Point', 'coordinates': [-73.9964312,..."
4,37.506885,-122.247573,POINT (-122.247573 37.506885),"{'type': 'Point', 'coordinates': [-122.247573,..."


Exportamos a json para incluirla como nueva colección en mongodb

In [10]:
cleanData.to_json("./inputs/cleaned_offices.json", orient="records")

# In bash: 
# $ mongoimport --db companies --collection cleancompanies --jsonArray --drop inputs/cleaned_offices.json

# (le estamos indicando que queremos importar nuestro json a la db companies con nombre cleancompanies)


#### Para visualizar los datos mediante mapas en Mongo Compass, iremos a nuestra nueva colección, pestaña Indexes > Create Index > nombre (p.ej. "geoindex") > Seleccionamos columna "location" y '2dsphere' en el desplegable